https://www.youtube.com/watch?v=TSyjBZKL5XA

In [1]:
# See: https://github.com/ipython/ipyparallel

In [2]:
# conda install faker
# If needed: Install findspark to make available spark from jupyter-python

In [92]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField, StringType, DateType
from pyspark.sql import functions as F


import logging

import pandas as pd
from faker import Faker
import random
from datetime import datetime


In [2]:
#conda update pyspark

In [3]:
#!pyspark --version

In [4]:
#spark.stop()

catalog_nm = "iceberg_catalog"
warehouse_path = "warehouse_loc"

conf = (
    pyspark.SparkConf()
    .setAppName("iceberg_with_filesystem")
    #pachages
    .set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.5.2")  # Corrected coordinate
    #SQL Extensions
    .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions:IcebergSparkSessionExtensions')
    #Configuring catalog
    .set(f'spark.sql.catalog.{catalog_nm}','org.apache.iceberg.spark.SparkCatalog')
    .set(f'spark.sql.catalog.{catalog_nm}.type', 'hadoop')
    .set(f'spark.sql.catalog.{catalog_nm}.warehouse', warehouse_path)
)

#Start SparkSession
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

In [5]:
# Set up a logger for your application
logging.basicConfig(
    level=logging.INFO,  # You can set to DEBUG, INFO, WARNING, ERROR, or CRITICAL
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Log format
    handlers=[logging.StreamHandler()]  # Output logs to the console
)

# Create a logger object for your application
logger = logging.getLogger(__name__)

## Improved conf:

In [6]:
catalog_nm = "iceberg_catalog" 
warehouse_path = "warehouse_loc"

# Configure Spark session for Iceberg
conf = (
    pyspark.SparkConf()
    .setAppName("iceberg_with_filesystem")
    .set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.5.2")
    .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
    .set(f'spark.sql.catalog.{catalog_nm}', 'org.apache.iceberg.spark.SparkCatalog')
    .set(f'spark.sql.catalog.{catalog_nm}.type', 'hadoop')
    .set(f'spark.sql.catalog.{catalog_nm}.warehouse', warehouse_path)
)

# Start SparkSession
spark = SparkSession.builder.config(conf=conf).getOrCreate()
logger.info("Spark session is up and running!")

# Adjust log level to suppress warnings
spark.sparkContext.setLogLevel("ERROR")


24/10/22 21:50:57 WARN Utils: Your hostname, MacBook-Pro-de-Victor.local resolves to a loopback address: 127.0.0.1; using 192.168.1.141 instead (on interface en0)
24/10/22 21:50:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/victorgalan/miniconda3/envs/iceberg_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/victorgalan/.ivy2/cache
The jars for the packages stored in: /Users/victorgalan/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fd851c7c-0353-4a73-8606-bfef895edd10;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.5.2 in central
:: resolution report :: resolve 190ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: o

In [7]:
#Run a query
database_nm = "my_database"
table_nm = "my_part_table"

my_table = f"{catalog_nm}.{database_nm}.{table_nm}"

In [9]:
spark.sql(f"""
            CREATE OR REPLACE TABLE {my_table} (
            name STRING,
            surname STRING,
            country STRING
            ) 
            using iceberg
            PARTITIONED BY (birthday DATE);
            """
         ).show()

++
||
++
++



In [11]:
#spark

In [11]:
def fake_data(num_records_to_create):
    num_inserts = num_records_to_create
    date_range = (-10, -1)  # Años desde el presente
    
    # Lista de países (puedes usar faker.country() si prefieres)
    #countries = ['United States', 'Canada', 'United Kingdom', ...]
    
    # Generar datos aleatorios
    fake = Faker()
    data = []
    for _ in range(num_inserts):
        data.append({
            'name': fake.name(),
            'surname': fake.last_name(),
            'birthday': datetime.strptime(fake.date_between(start_date=f'{date_range[0]}y', end_date=f'{date_range[1]}y').strftime('%Y-%m-%d'), '%Y-%m-%d'),
            #'country': str(random.choice(countries))  # O str(fake.random_element(elements=countries))
            'country': fake.country() 
        })
    return data

In [13]:
data = fake_data(2)
data

[{'name': 'Shelly Guzman',
  'surname': 'Todd',
  'birthday': datetime.datetime(2019, 10, 16, 0, 0),
  'country': 'Samoa'},
 {'name': 'Mr. Jeremy Riley',
  'surname': 'Howard',
  'birthday': datetime.datetime(2020, 4, 29, 0, 0),
  'country': 'Tajikistan'}]

In [67]:
def insert_fake_data_into_table(num_records_to_create):
    data = fake_data(num_records_to_create)
    print(data)
    # Create DataFrame from valid data
    df = spark.createDataFrame(data)
    df = df.withColumn("birthday", df["birthday"].cast("date"))
    
    df.createOrReplaceTempView("fake_data")
    
    #df.printSchema()
    #df.show()
    
    logger.info(f"Number of records to be inserted: {df.count()}")
    query_view = f"""
    SELECT * 
    FROM fake_data
    LIMIT 10;
    """
    spark.sql(query_view).show()

    query = f"""
    INSERT INTO {my_table}
    SELECT name, 
            surname, 
            country, 
            to_date(birthday, 'yyyy-MM-dd') AS birthday
    FROM fake_data;
    """
    
    spark.sql(query).show()
    logger.info(f"Records inserted successfully!!")


In [68]:
insert_fake_data_into_table(1)

[{'name': 'Matthew Cook', 'surname': 'Green', 'birthday': datetime.datetime(2020, 2, 20, 0, 0), 'country': 'Cuba'}]


2024-10-22 22:25:41,278 - __main__ - INFO - Number of records to be inserted: 1


+----------+-------+------------+-------+
|  birthday|country|        name|surname|
+----------+-------+------------+-------+
|2020-02-20|   Cuba|Matthew Cook|  Green|
+----------+-------+------------+-------+



2024-10-22 22:25:42,232 - __main__ - INFO - Records inserted successfully!!


++
||
++
++



In [ ]:
#Insert into with dataframe api

#df = spark.createDataFrame([Row(name="Sr. victor",surname="galan")])
#df.writeTo(f"{catalog_nm}.{table_nm}").append()

In [20]:
def query_my_table():
    #Query the table
    spark.sql(
        f"""SELECT * 
        FROM {my_table};
        """
    ).show()

In [21]:
query_my_table()

+----------------+-------+----------+----------+
|            name|surname|   country|  birthday|
+----------------+-------+----------+----------+
|   Shelly Guzman|   Todd|     Samoa|2019-10-16|
|Mr. Jeremy Riley| Howard|Tajikistan|2020-04-29|
+----------------+-------+----------+----------+



In [22]:
# Create a new database in the catalog
try:
    spark.sql(f"CREATE DATABASE {catalog_nm}.{table_nm}")
except Exception as e:
    logging.error(f"Error: {e}")

In [20]:
#spark.catalog.clearCache()

In [23]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [33]:
def describe_extended():
    describe_extended_pyspark_df = spark.sql(f"DESCRIBE EXTENDED {my_table}")
    #describe_extended_pyspark_df.show(truncate=False)
    
    # Convert the Spark DataFrame back to a pandas DataFrame using Arrow
    describe_extended_df = describe_extended_pyspark_df.select("*").toPandas()
    return describe_extended_df

In [119]:
describe_extended_df = describe_extended()
describe_extended_df

,col_name,data_type,comment
0,name,string,None
1,surname,string,None
2,country,string,None
3,birthday,date,None
4,,,
5,# Metadata Columns,,
6,_spec_id,int,
7,_partition,struct<birthday:date>,
8,_file,string,
9,_pos,bigint,


In [35]:
query_my_table()

+----------------+-------+----------+----------+
|            name|surname|   country|  birthday|
+----------------+-------+----------+----------+
|   Shelly Guzman|   Todd|     Samoa|2019-10-16|
|Mr. Jeremy Riley| Howard|Tajikistan|2020-04-29|
+----------------+-------+----------+----------+



In [93]:
data = [("jose", "garcia", "mexico", "2020-03-03"),
    ("pedro", "gonzalez", "brazil","2022-03-03")]

schema = StructType([ \
    StructField("name",StringType(),True),
    StructField("surname",StringType(),True),
    StructField("country",StringType(),True),
    StructField("birthday", StringType(), True),
  ])

df = spark.createDataFrame(data=data,schema=schema)
df = df.withColumn("birthday", F.col("birthday").cast(DateType()))
df.printSchema()
df.show(truncate=False)


root
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- birthday: date (nullable = true)

+-----+--------+-------+----------+
|name |surname |country|birthday  |
+-----+--------+-------+----------+
|jose |garcia  |mexico |2020-03-03|
|pedro|gonzalez|brazil |2022-03-03|
+-----+--------+-------+----------+



In [142]:
def add_one_partition_column(my_table, col_str):
    query_evo = f'''
    ALTER TABLE {my_table}
    ADD PARTITION FIELD {col_str}
    '''
    spark.sql(query_evo)
    logging.info(f"Patition added {col_str}")


In [143]:
add_one_partition_column(my_table, "birthday")

2024-10-22 23:06:57,034 - root - INFO - Patition added birthday


In [164]:
data_to_overwrite_df = spark.sql(f"SELECT * FROM {my_table}")
data_to_overwrite_df.show()

+----------------+-------+-------------+----------+
|            name|surname|      country|  birthday|
+----------------+-------+-------------+----------+
|     Heidi Evans| Miller|      Nigeria|2016-08-24|
|   Mark Johnston| Melton|New Caledonia|2019-08-04|
|   Shelly Guzman|   Todd|        Samoa|2019-10-16|
|    Matthew Cook|  Green|         Cuba|2020-02-20|
|Mr. Jeremy Riley| Howard|   Tajikistan|2020-04-29|
|   Julie Mueller|  Smith|      Ireland|2020-09-21|
+----------------+-------+-------------+----------+



In [165]:
data_to_overwrite_df.show()

+----------------+-------+-------------+----------+
|            name|surname|      country|  birthday|
+----------------+-------+-------------+----------+
|     Heidi Evans| Miller|      Nigeria|2016-08-24|
|   Mark Johnston| Melton|New Caledonia|2019-08-04|
|   Shelly Guzman|   Todd|        Samoa|2019-10-16|
|    Matthew Cook|  Green|         Cuba|2020-02-20|
|Mr. Jeremy Riley| Howard|   Tajikistan|2020-04-29|
|   Julie Mueller|  Smith|      Ireland|2020-09-21|
+----------------+-------+-------------+----------+



In [163]:
(
    #data_to_overwrite_df.writeTo(my_table).partitionedBy("birthday").overwritePartitions()
    data_to_overwrite_df.distinct().writeTo(my_table).overwrite(F.lit(True))
)

In [147]:
data_to_overwrite_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- birthday: date (nullable = true)



In [170]:
# Change the sort order using SQL
spark.sql(f"""
    ALTER TABLE {my_table}
    WRITE ORDERED BY (birthday ASC NULLS LAST, name DESC NULLS FIRST)
""")

# Verify the change (optional)
spark.sql(f"DESCRIBE {my_table}").show(truncate=False)

+-----------------------+---------+-------+
|col_name               |data_type|comment|
+-----------------------+---------+-------+
|name                   |string   |null   |
|surname                |string   |null   |
|country                |string   |null   |
|birthday               |date     |null   |
|# Partition Information|         |       |
|# col_name             |data_type|comment|
|birthday               |date     |null   |
+-----------------------+---------+-------+



# VACUUM

In [200]:
60*25

1500

In [204]:
# Vacuum command to clean up old files
# Set vacuum properties for the Iceberg table
spark.sql(f"""
    ALTER TABLE {catalog_nm}.{database_nm}.{table_nm}
    SET TBLPROPERTIES (
        'vacuum_max_snapshot_age_seconds' = {60*5},  -- 1 hour
        'vacuum_min_snapshots_to_keep' = '1'
    )
""")


DataFrame[]

In [205]:
# Snapshots can be queried in a spark session
snapshots = spark.sql(
    f"""
    SELECT * FROM {my_table}.snapshots;
    """
).show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2024-10-22 21:57:...|5917362355917637182|               null|   append|warehouse_loc/my_...|{spark.app.id -> ...|
|2024-10-22 22:23:...|2023042709182525745|5917362355917637182|   append|warehouse_loc/my_...|{spark.app.id -> ...|
|2024-10-22 22:24:...|2928266320472832784|2023042709182525745|   append|warehouse_loc/my_...|{spark.app.id -> ...|
|2024-10-22 22:25:...|3503764429907316250|2928266320472832784|   append|warehouse_loc/my_...|{spark.app.id -> ...|
|2024-10-22 22:25:...| 401056988078697897|3503764429907316250|   append|warehouse_loc/my_...|{spark.app.id -> ...|
|2024-10-22 22:54:...|8819043496371456546| 401056988078697897|overwrite|warehous

# OPTIMIZE

In [199]:
# Optimize the Iceberg table to clean up unused files
spark.sql(f"""OPTIMIZE {catalog_nm}.{database_nm}.{table_nm} REWRITE DATA USING BIN_PACK""")

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'OPTIMIZE'.(line 1, pos 0)

== SQL ==
OPTIMIZE iceberg_catalog.my_database.my_part_table REWRITE DATA USING BIN_PACK
^^^


# compaction

In [206]:
database_nm

'my_database'

In [219]:
spark.sql(f"SHOW TABLES IN {catalog_nm}.{database_nm}").show()


+-----------+-------------+-----------+
|  namespace|    tableName|isTemporary|
+-----------+-------------+-----------+
|my_database|my_part_table|      false|
+-----------+-------------+-----------+



In [232]:
spark.sql(f"""
    CALL {catalog_nm}.system.rewrite_data_files(
        table => '{database_nm}.{table_nm}', 
        strategy => 'sort', 
        sort_order => 'zorder(birthday, name)',
        options => map('min-file-size-bytes', '5242880', 'max-file-size-bytes', '1073741824')
    )
""").show()

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|                         0|                     0|                    0|                      0|
+--------------------------+----------------------+---------------------+-----------------------+



In [234]:
spark.sql(f"""
    CALL {catalog_nm}.system.rewrite_data_files(
        table => '{database_nm}.{table_nm}', 
        strategy => 'sort', 
        sort_order => 'zorder(birthday, name)',
        options => map('min-input-files', '5', 'max-concurrent-file-group-rewrites', '10')
    )
""").show()

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|                         0|                     0|                    0|                      0|
+--------------------------+----------------------+---------------------+-----------------------+



In [233]:
spark.sql(f"SELECT * FROM {catalog_nm}.system.files('{database_nm}.{table_nm}')").show()


ParseException: 
[INVALID_SQL_SYNTAX] Invalid SQL syntax: table valued function cannot specify database name: `iceberg_catalog`.`system`.`files`.(line 1, pos 14)

== SQL ==
SELECT * FROM iceberg_catalog.system.files('my_database.my_part_table')
--------------^^^


In [226]:
# Rewrite data files for a specific partition
spark.sql(f"""
    CALL {catalog_nm}.system.rewrite_data_files(
        table => '{database_nm}.{my_table}', 
        strategy => 'binpack',
        options => map('min-file-size-bytes', '5242880', 'max-file-size-bytes', '1073741824'),
        partition => map('partition_column', 'partition_value')
    )
""")

AnalysisException: Could not build name to arg map: Unknown argument: partition

In [148]:
spark.sql(f"SHOW CREATE TABLE {my_table}").show(truncate=False)


In [149]:
describe_extended_df = describe_extended()
describe_extended_df[describe_extended_df["col_name"] == "_partition"]

,col_name,data_type,comment
10,_partition,struct<birthday:date>,


In [97]:
#%whos

In [66]:
insert_fake_data_into_table(1)

2024-10-22 22:25:02,632 - __main__ - INFO - Number of records to be inserted: 1


+----------+-------+-------------+-------+
|  birthday|country|         name|surname|
+----------+-------+-------------+-------+
|2020-09-21|Ireland|Julie Mueller|  Smith|
+----------+-------+-------------+-------+



2024-10-22 22:25:03,594 - __main__ - INFO - Records inserted successfully!!


++
||
++
++



In [64]:
query_my_table()

+----------------+-------+-------------+----------+
|            name|surname|      country|  birthday|
+----------------+-------+-------------+----------+
|     Heidi Evans| Miller|      Nigeria|2016-08-24|
|   Mark Johnston| Melton|New Caledonia|2019-08-04|
|   Shelly Guzman|   Todd|        Samoa|2019-10-16|
|Mr. Jeremy Riley| Howard|   Tajikistan|2020-04-29|
+----------------+-------+-------------+----------+



In [138]:
def drop_one_partition_column(my_table, col_str):
    query_evo = f'''
    ALTER TABLE {my_table}
    DROP PARTITION FIELD `{col_str}`
    '''
    spark.sql(query_evo)
    logging.info(f"Patition dropped {col_str}")



In [139]:
drop_one_partition_column(my_table, "birthday")

IllegalArgumentException: Cannot find partition field to remove: ref(name="birthday")

In [78]:
describe_extended_df = describe_extended()
describe_extended_df[describe_extended_df["col_name"] == "_partition"]

,col_name,data_type,comment
7,_partition,struct<birthday:date>,


In [153]:
partition_info = spark.sql(f"SELECT * FROM {my_table}.partitions")
partition_info.show()


+------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|   partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+------------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|{2020-02-20}|      0|           4|         1|                         1380|                           0|                         0|                           0|                         0|2024-10-22 23:07:...|     1752009770909895026|
|{2019-10-16}|      0|           4|         1|              

In [152]:
query_my_table()

+----------------+-------+-------------+----------+
|            name|surname|      country|  birthday|
+----------------+-------+-------------+----------+
|   Mark Johnston| Melton|New Caledonia|2019-08-04|
|     Heidi Evans| Miller|      Nigeria|2016-08-24|
|    Matthew Cook|  Green|         Cuba|2020-02-20|
|   Julie Mueller|  Smith|      Ireland|2020-09-21|
|   Shelly Guzman|   Todd|        Samoa|2019-10-16|
|Mr. Jeremy Riley| Howard|   Tajikistan|2020-04-29|
|   Mark Johnston| Melton|New Caledonia|2019-08-04|
|     Heidi Evans| Miller|      Nigeria|2016-08-24|
|    Matthew Cook|  Green|         Cuba|2020-02-20|
|   Julie Mueller|  Smith|      Ireland|2020-09-21|
|   Shelly Guzman|   Todd|        Samoa|2019-10-16|
|Mr. Jeremy Riley| Howard|   Tajikistan|2020-04-29|
|   Mark Johnston| Melton|New Caledonia|2019-08-04|
|     Heidi Evans| Miller|      Nigeria|2016-08-24|
|    Matthew Cook|  Green|         Cuba|2020-02-20|
|   Julie Mueller|  Smith|      Ireland|2020-09-21|
|   Shelly G

In [151]:
# Replace `my_table` and `birthday` with your actual table name and partition column
null_partition_data = spark.sql(f"SELECT * FROM {my_table} WHERE birthday IS NULL")
null_partition_data.show()


+----+-------+-------+--------+
|name|surname|country|birthday|
+----+-------+-------+--------+
+----+-------+-------+--------+



In [56]:
query_evo = f'''
ALTER TABLE {my_table}
ADD PARTITION FIELD country
'''
spark.sql(query_evo)


DataFrame[]

In [57]:
spark.sql(f"""
SELECT *
FROM {my_table}
LIMIT 10;
""").show()

+-------------+-------+--------------------+----------+
|         name|surname|             country|  birthday|
+-------------+-------+--------------------+----------+
|    Joseph Ho| Castro|Falkland Islands ...|2019-09-01|
|Benjamin Wong| Tucker|             Ireland|2020-09-03|
+-------------+-------+--------------------+----------+



In [62]:
query_evo = f'''
ALTER TABLE {catalog_nm}.{table_nm}
ADD PARTITION FIELD truncate(4, birthday) as year_birthday
'''
spark.sql(query_evo)


Py4JJavaError: An error occurred while calling o37.sql.
: org.apache.iceberg.exceptions.ValidationException: Cannot bind: truncate[4] cannot transform date values from 'birthday'
	at org.apache.iceberg.exceptions.ValidationException.check(ValidationException.java:49)
	at org.apache.iceberg.expressions.UnboundTransform.bind(UnboundTransform.java:48)
	at org.apache.iceberg.expressions.UnboundTransform.bind(UnboundTransform.java:25)
	at org.apache.iceberg.BaseUpdatePartitionSpec.resolve(BaseUpdatePartitionSpec.java:337)
	at org.apache.iceberg.BaseUpdatePartitionSpec.addField(BaseUpdatePartitionSpec.java:174)
	at org.apache.iceberg.BaseUpdatePartitionSpec.addField(BaseUpdatePartitionSpec.java:44)
	at org.apache.spark.sql.execution.datasources.v2.AddPartitionFieldExec.run(AddPartitionFieldExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec

In [83]:
spark.sql(f"DESCRIBE EXTENDED {my_table}").show(truncate=False)


+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                            |comment|
+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|name                        |string                                                                                                               |null   |
|surname                     |string                                                                                                               |null   |
|country                     |string                                                                                                               |null   |
|birthday                    |date                        

In [38]:
spark.sql(f"""
SELECT *
FROM {my_table}
LIMIT 10;
""").show()

+---------------+-------+--------------------+----------+
|           name|surname|             country|  birthday|
+---------------+-------+--------------------+----------+
| Anthony Henson| Campos|              Zambia|2016-08-26|
|   Daniel James|  Davis|Cocos (Keeling) I...|2017-03-10|
|   Jason Graves|Raymond|              Jordan|2017-05-28|
|   Molly Knight|Edwards|                Togo|2018-09-20|
|Amanda Garrison|  White|       Liechtenstein|2022-05-28|
+---------------+-------+--------------------+----------+



In [58]:
query = f"""
INSERT INTO {my_table}
SELECT name, 
        surname, 
        country, 
        to_date(birthday, 'yyyy-MM-dd') AS birthday
FROM fake_data;
"""

spark.sql(query).show()

++
||
++
++



In [59]:
spark.sql(f"DESCRIBE EXTENDED {my_table}").show(truncate=False)

+----------------------------+-----------------------------------------+-------+
|col_name                    |data_type                                |comment|
+----------------------------+-----------------------------------------+-------+
|name                        |string                                   |null   |
|surname                     |string                                   |null   |
|country                     |string                                   |null   |
|birthday                    |date                                     |null   |
|# Partition Information     |                                         |       |
|# col_name                  |data_type                                |comment|
|country                     |string                                   |null   |
|                            |                                         |       |
|# Metadata Columns          |                                         |       |
|_spec_id                   


query_evo = f"""
ALTER TABLE {catalog_nm}.{table_nm}
SET TBLPROPERTIES (
  'format-version' = '2'
)"""
spark.sql(query_evo)


In [50]:
num_inserts = 1
date_range = (-40, -20)  # Años desde el presente

# Lista de países (puedes usar faker.country() si prefieres)
countries = ['United States', 'Canada', 'United Kingdom', ...]

# Generar datos aleatorios
fake = Faker()
data = []
for _ in range(num_inserts):
    data.append({
        'name': fake.name(),
        'surname': fake.last_name(),
        'birthday': datetime.strptime(fake.date_between(start_date=f'{date_range[0]}y', end_date=f'{date_range[1]}y').strftime('%Y-%m-%d'), '%Y-%m-%d'),
        'country': str(random.choice(countries))  # O str(fake.random_element(elements=countries))
    })

In [51]:
data

[{'name': 'Joseph Ellis',
  'surname': 'Garcia',
  'birthday': datetime.datetime(1989, 8, 16, 0, 0),
  'country': 'Ellipsis'}]

In [52]:
# Crear un DataFrame de PySpark
df = spark.createDataFrame(data)

# Create DataFrame from valid data
df = spark.createDataFrame(data)
df = df.withColumn("birthday", df["birthday"].cast("date"))

df.createOrReplaceTempView("fake_data")

df.printSchema()
df.show()

root
 |-- birthday: date (nullable = true)
 |-- country: string (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)

+----------+--------+------------+-------+
|  birthday| country|        name|surname|
+----------+--------+------------+-------+
|1989-08-16|Ellipsis|Joseph Ellis| Garcia|
+----------+--------+------------+-------+



In [53]:
query_view = f"""
SELECT * 
FROM fake_data
LIMIT 10;
"""

spark.sql(query_view).show()

+----------+--------+------------+-------+
|  birthday| country|        name|surname|
+----------+--------+------------+-------+
|1989-08-16|Ellipsis|Joseph Ellis| Garcia|
+----------+--------+------------+-------+



In [61]:
query = f"""
INSERT INTO {my_table}
SELECT name, 
        surname, 
        country, 
        to_date(birthday, 'yyyy-MM-dd') AS birthday
FROM fake_data;
"""

spark.sql(query).show()

++
||
++
++



In [55]:
#Insert into with dataframe api

#df = spark.createDataFrame([Row(name="Sr. victor",surname="galan")])
#df.writeTo(f"{catalog_nm}.{table_nm}").append()

In [62]:
#Query the table
spark.sql(
    f"""SELECT * 
    FROM {my_table};
    """
).show()

+-------------+-------+--------------------+----------+
|         name|surname|             country|  birthday|
+-------------+-------+--------------------+----------+
|    Joseph Ho| Castro|Falkland Islands ...|2019-09-01|
|Benjamin Wong| Tucker|             Ireland|2020-09-03|
|    Joseph Ho| Castro|Falkland Islands ...|2019-09-01|
|Benjamin Wong| Tucker|             Ireland|2020-09-03|
|    Joseph Ho| Castro|Falkland Islands ...|2019-09-01|
|Benjamin Wong| Tucker|             Ireland|2020-09-03|
+-------------+-------+--------------------+----------+



In [64]:
#Query the table
spark.sql(
    f"""SELECT * 
    FROM {my_table}.history;
    """
).show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2024-10-21 22:05:39.118|756897460753581554 |null               |false              |
|2024-10-21 22:19:12.081|6660628102038648577|null               |true               |
|2024-10-21 22:23:51.307|3516254222102115996|6660628102038648577|true               |
|2024-10-21 22:26:11.313|3329146188426717797|3516254222102115996|true               |
+-----------------------+-------------------+-------------------+-------------------+



# Tests

In [247]:
df = spark.sql(
    f"""SELECT * 
    FROM {my_table};
    """
)
df.show()

+----------------+-------+-------------+----------+
|            name|surname|      country|  birthday|
+----------------+-------+-------------+----------+
|     Heidi Evans| Miller|      Nigeria|2016-08-24|
|   Mark Johnston| Melton|New Caledonia|2019-08-04|
|   Shelly Guzman|   Todd|        Samoa|2019-10-16|
|    Matthew Cook|  Green|         Cuba|2020-02-20|
|Mr. Jeremy Riley| Howard|   Tajikistan|2020-04-29|
|   Julie Mueller|  Smith|      Ireland|2020-09-21|
+----------------+-------+-------------+----------+



In [240]:
(
    df.withColumn("array", F.array_distinct(F.array(F.col("name"),F.col("surname"))))
).show(truncate=False)

+----------------+-------+-------------+----------+--------------------------+
|name            |surname|country      |birthday  |array                     |
+----------------+-------+-------------+----------+--------------------------+
|Heidi Evans     |Miller |Nigeria      |2016-08-24|[Heidi Evans, Miller]     |
|Mark Johnston   |Melton |New Caledonia|2019-08-04|[Mark Johnston, Melton]   |
|Shelly Guzman   |Todd   |Samoa        |2019-10-16|[Shelly Guzman, Todd]     |
|Matthew Cook    |Green  |Cuba         |2020-02-20|[Matthew Cook, Green]     |
|Mr. Jeremy Riley|Howard |Tajikistan   |2020-04-29|[Mr. Jeremy Riley, Howard]|
|Julie Mueller   |Smith  |Ireland      |2020-09-21|[Julie Mueller, Smith]    |
+----------------+-------+-------------+----------+--------------------------+



In [242]:
# Create the array column
df = df.withColumn("array", F.array_distinct(F.array(F.col("name"), F.col("surname"))))

# Filter out elements from 'array' that match the value in 'name'
df = df.withColumn(
    "filtered_array",
    F.expr("filter(array, x -> x != name)")
)

# Show the result
df.show(truncate=False)


+----------------+-------+-------------+----------+--------------------------+--------------+
|name            |surname|country      |birthday  |array                     |filtered_array|
+----------------+-------+-------------+----------+--------------------------+--------------+
|Heidi Evans     |Miller |Nigeria      |2016-08-24|[Heidi Evans, Miller]     |[Miller]      |
|Mark Johnston   |Melton |New Caledonia|2019-08-04|[Mark Johnston, Melton]   |[Melton]      |
|Shelly Guzman   |Todd   |Samoa        |2019-10-16|[Shelly Guzman, Todd]     |[Todd]        |
|Matthew Cook    |Green  |Cuba         |2020-02-20|[Matthew Cook, Green]     |[Green]       |
|Mr. Jeremy Riley|Howard |Tajikistan   |2020-04-29|[Mr. Jeremy Riley, Howard]|[Howard]      |
|Julie Mueller   |Smith  |Ireland      |2020-09-21|[Julie Mueller, Smith]    |[Smith]       |
+----------------+-------+-------------+----------+--------------------------+--------------+



In [244]:

# Create the array column
df = df.withColumn("array", F.array_distinct(F.array(F.col("name"), F.col("surname"))))

# Filter out elements from 'array' that match the value in 'name'
# Then join the filtered array elements into a single string, separated by a comma
df = df.withColumn(
    "filtered_string",
    F.expr("array_join(filter(array, x -> x != name), ', ')")
)

# Show the result
df.show(truncate=False)


+----------------+-------+-------------+----------+--------------------------+--------------+----------------+
|name            |surname|country      |birthday  |array                     |filtered_array|filtered_string |
+----------------+-------+-------------+----------+--------------------------+--------------+----------------+
|Heidi Evans     |Miller |Nigeria      |2016-08-24|[Heidi Evans, Miller]     |[Miller]      |Heidi Evans     |
|Mark Johnston   |Melton |New Caledonia|2019-08-04|[Mark Johnston, Melton]   |[Melton]      |Mark Johnston   |
|Shelly Guzman   |Todd   |Samoa        |2019-10-16|[Shelly Guzman, Todd]     |[Todd]        |Shelly Guzman   |
|Matthew Cook    |Green  |Cuba         |2020-02-20|[Matthew Cook, Green]     |[Green]       |Matthew Cook    |
|Mr. Jeremy Riley|Howard |Tajikistan   |2020-04-29|[Mr. Jeremy Riley, Howard]|[Howard]      |Mr. Jeremy Riley|
|Julie Mueller   |Smith  |Ireland      |2020-09-21|[Julie Mueller, Smith]    |[Smith]       |Julie Mueller   |
+

In [250]:

# Create the array column with distinct values and filter out nulls
df = df.withColumn("array", 
                   F.expr("filter(array_distinct(array(name, surname)), x -> x IS NOT NULL)"))

# Remove elements from 'array' that match the value in 'name'
filtered_array = F.expr("filter(array, x -> x != name)")

# Add the filtered array back to the DataFrame
df = df.withColumn("filtered_array", filtered_array)

# Join the filtered array into a single string
df = df.withColumn("filtered_string", F.concat_ws(", ", F.col("filtered_array")))

# Show the result
df.show(truncate=False)


+----------------+-------+-------------+----------+--------------------------+--------------+---------------+
|name            |surname|country      |birthday  |array                     |filtered_array|filtered_string|
+----------------+-------+-------------+----------+--------------------------+--------------+---------------+
|Heidi Evans     |Miller |Nigeria      |2016-08-24|[Heidi Evans, Miller]     |[Miller]      |Miller         |
|Mark Johnston   |Melton |New Caledonia|2019-08-04|[Mark Johnston, Melton]   |[Melton]      |Melton         |
|Shelly Guzman   |Todd   |Samoa        |2019-10-16|[Shelly Guzman, Todd]     |[Todd]        |Todd           |
|Matthew Cook    |Green  |Cuba         |2020-02-20|[Matthew Cook, Green]     |[Green]       |Green          |
|Mr. Jeremy Riley|Howard |Tajikistan   |2020-04-29|[Mr. Jeremy Riley, Howard]|[Howard]      |Howard         |
|Julie Mueller   |Smith  |Ireland      |2020-09-21|[Julie Mueller, Smith]    |[Smith]       |Smith          |
+---------